# Konvertering av Bergens registeret



In [1]:
from hentOgVaskData import hent_og_vask_data
import polars as pl
pl.Config.set_tbl_rows(30)

polars.config.Config

In [2]:
# leser inn Data exportert i fra Bergens Access database
# I fra den tabellen som heter "tblInnmeldte"
df = hent_og_vask_data()
# Medlems sjekk, det er viktig at dette tallet er nokså likt som rapportert in til statsforvalteren 282
print("Antall medlemmer i Bergens registeret -->", df.filter(pl.col("Kategori").str.starts_with("M"))["MedlemID"])
print("Alle data i fra excel fil -->", df)

Could not determine dtype for column 43, falling back to string


Antall medlemmer i Bergens registeret --> shape: (281,)
Series: 'MedlemID' [i64]
[
	14570
	15536
	14800
	14720
	13073
	12181
	14015
	15442
	14791
	11208
	13277
	15537
	12968
	14462
	11209
	…
	11692
	13649
	14744
	12506
	14482
	14481
	14484
	14483
	14063
	13622
	14432
	11027
	14294
	13451
	15638
]
Alle data i fra excel fil --> shape: (582, 46)
┌──────────┬────────────┬────────────┬──────────┬───┬───────────┬──────────┬───────────┬───────────┐
│ MedlemID ┆ Innmelding ┆ Er den     ┆ Menighet ┆ … ┆ UtmeldtDa ┆ DødsDato ┆ Kommune   ┆ KID-nr    │
│ ---      ┆ sdato      ┆ innmeldte  ┆ ---      ┆   ┆ to        ┆ ---      ┆ ---       ┆ ---       │
│ i64      ┆ ---        ┆ døpt ?     ┆ str      ┆   ┆ ---       ┆ str      ┆ str       ┆ str       │
│          ┆ date       ┆ ---        ┆          ┆   ┆ date      ┆          ┆           ┆           │
│          ┆            ┆ bool       ┆          ┆   ┆           ┆          ┆           ┆           │
╞══════════╪════════════╪════════════╪══════════╪

In [3]:
'''
df.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"
         ).sort("SamID"
               )#.filter(pl.col("Personnummer").str.len_chars() == 6)
'''

'\ndf.select("MedlemID", "SamID", "FORNAVN", "EFTERNAVN", "Personnummer"\n         ).sort("SamID"\n               )#.filter(pl.col("Personnummer").str.len_chars() == 6)\n'

## Rekkefølge på konverteringen
- Alle som har fødselsnummer, medlemmer med flere
- Alle som er registrert med "MedlemID", "SamID", "MorID", "FarID"
- Alle som har "MedlemID" i "SamID", "MorID", "FarID"
- 
- Det vil være overlappende "MedlemID" for de som er listet over. Få ut en liste og krymp den til disticte verdier av "MedlemID"
- Resterende personer vil ha datakvalitet av varierende kvalitet. Disse konverteres til slutt. Det er resterende som ikke er med i den oppbyggde "MedlemID" listen. Ingen av disse er medlemmer. De har heller ikke forbindelser til andre personer.

### Denne er ikke nødvendig, så lenge vi gjør den siste testen ved henting av data fra excel filen
``` python
df_medlemId = (
    df
    .filter(
        (pl.col("Kategori").str.starts_with("M")) &
        # Vet ikke hvorfor dette blir gjort, men det er det som skjer i access databasen "qryMedlemmer"
        (pl.col("POSTNR").is_between(pl.lit("100"), pl.lit("9000"), closed="both"))
    )
    .select("MedlemID")
)
df_medlemId
```

In [4]:

# Tror ikke dette blir nødvendig. Personers forbindelser blir oppdatert etter at alle
# relevante personer er lagt inn i egen ferdig vasket tabell under "df_connectedMedlemer"

# Oppdaterer df og setter alle ikke eksisterende kontakter til null
#df = df.with_columns([
#    pl.when(pl.col(c).is_in(missing_ids["MedlemID"].to_list()))
#      .then(None)
#      .otherwise(pl.col(c))
#      .alias(c)
#    for c in ["SamID", "MorID", "FarID"]
#])


# Konvertering utrekk

In [5]:
# Leser inn hjelpe tabeller for Medlemstyper og Sivilstatus
df_sivil = pl.read_excel("sivil_status.xlsx")
df_medlems_type = pl.read_excel("kategorier.xlsx")

print(df_medlems_type)
print(df_sivil)

shape: (13, 3)
┌──────────────────┬──────────────┬─────────────────────────────────┐
│ Access Kattegori ┆ Cornerstone  ┆ Beskirvelse                     │
│ ---              ┆ ---          ┆ ---                             │
│ str              ┆ str          ┆ str                             │
╞══════════════════╪══════════════╪═════════════════════════════════╡
│ DB               ┆ null         ┆ ? dåpsbarn                      │
│ DF               ┆ null         ┆ ? dåps foreldre                 │
│ DØD              ┆ null         ┆ Blir satt til status død, egen… │
│ FLY              ┆ null         ┆ ? Tror det betyr flyttet, kan … │
│ M                ┆ Medlem       ┆ null                            │
│ MB               ┆ Medlem       ┆ null                            │
│ MS               ┆ Støttemedlem ┆ null                            │
│ MU               ┆ Medlem       ┆ Ungdomsmedlemmer blir satt til… │
│ O                ┆ null         ┆ Organisasjon, Egen behandling   │
│ P  

In [6]:
DATE_FORMAT :str = "%d.%m.%Y"
konverterings_tabell = (
    df
        .join(
            # 1. First join: rename "Cornerstone" to be specific
            df_medlems_type.select([
                "Access Kattegori", 
                pl.col("Cornerstone").alias("Medlemstype_Cornerstone")
            ]),
            left_on="Kategori",
            right_on="Access Kattegori",
            how="left"
        )
        .join(
            # 2. Second join: rename "Cornerstone" here as well
            df_sivil.select([
                "Access Sivilstatus", 
                pl.col("Cornerstone").alias("Sivilstand_Cornerstone")
            ]),
            left_on="Sivilstand",
            right_on="Access Sivilstatus",
            how="left"
        )
        .select(
        pl.col("MedlemID"), # 
        pl.col("Innmeldingsdato").dt.strftime(DATE_FORMAT),
        pl.col("Er den innmeldte døpt ?"),
        pl.col("Menighet"),
        pl.col("Fødested"),
        pl.col("Dåpsdato").dt.strftime(DATE_FORMAT),
        pl.col("Ant barn under 15 år"),
        pl.col("Dåpskirke/sted"),
        pl.col("Fødselsdato").dt.strftime(DATE_FORMAT),
        pl.col("Personnummer"),
        pl.col("EFTERNAVN"),
        pl.col("FORNAVN"),
        pl.col("Stilling"),
        pl.col("ADRESSE"),
        pl.col("E-postadresse"),
        pl.col("POSTNR"),
        pl.col("Og"),
        pl.col("SAM_FOR"),
        pl.col("SAM_EFTER"),
        pl.col("Utsendelse"),
        pl.col("Utsendelses kode"),
        pl.col("Telefon priv"),
        pl.col("Telefon mob"),
        pl.col("Sivilstand_Cornerstone").alias("Sivilstand"),
        #pl.col("Sivilstand"),
        pl.col("Trossamfunn"),
        pl.col("Gift?"),
        pl.col("Utsendelsesformat"),
        pl.col("Ektemakens trossamfunn"),
        pl.col("Anmerkning"),
        pl.col("Medlemstype_Cornerstone").alias("Kategori"),
        #pl.col("Kategori"),
        pl.col("Giro"),
        pl.col("RegDato").dt.strftime(DATE_FORMAT),
        pl.col("SamID"),
        pl.col("FarID"),
        pl.col("MorID"),
        pl.col("Kjønn"),
        pl.col("Svart?"),
        pl.col("Faddere"),
        pl.col("Er mor og far gift?"),
        pl.col("Prest"),
        pl.col("Dåpsattest"),
        pl.col("Søk"),
        pl.col("UtmeldtDato").dt.strftime(DATE_FORMAT),
        #pl.col("DødsDato").dt.strftime(DATE_FORMAT), Dødsdato er en string
        pl.col("DødsDato"),
        pl.col("Kommune"),
        pl.col("KID-nr")

    )
)#.filter(pl.col("DødsDato").is_not_null())

print(konverterings_tabell)
'''
enriched_sales = (
    sales
    .join(
        customers.select(["customer_id", "customer_name"]),
        on="customer_id",
        how="left"
    )
    .join(
        products.select(["product_id", "product_name"]),
        on="product_id",
        how="left"
    )
    .join(
        stores.select(["store_id", "store_name"]),
        on="store_id",
        how="left"
    )
    .select(
        "sale_id",
        "customer_name",
        "product_name",
        "store_name",
        "quantity"
    )
)
'''

shape: (582, 46)
┌──────────┬────────────┬────────────┬──────────┬───┬───────────┬──────────┬───────────┬───────────┐
│ MedlemID ┆ Innmelding ┆ Er den     ┆ Menighet ┆ … ┆ UtmeldtDa ┆ DødsDato ┆ Kommune   ┆ KID-nr    │
│ ---      ┆ sdato      ┆ innmeldte  ┆ ---      ┆   ┆ to        ┆ ---      ┆ ---       ┆ ---       │
│ i64      ┆ ---        ┆ døpt ?     ┆ str      ┆   ┆ ---       ┆ str      ┆ str       ┆ str       │
│          ┆ str        ┆ ---        ┆          ┆   ┆ str       ┆          ┆           ┆           │
│          ┆            ┆ bool       ┆          ┆   ┆           ┆          ┆           ┆           │
╞══════════╪════════════╪════════════╪══════════╪═══╪═══════════╪══════════╪═══════════╪═══════════╡
│ 13812    ┆ null       ┆ false      ┆ Bergen   ┆ … ┆ null      ┆ null     ┆ AURLAND   ┆           │
│ 14571    ┆ 30.09.2002 ┆ false      ┆ Bergen   ┆ … ┆ 15.08.201 ┆ null     ┆           ┆           │
│          ┆            ┆            ┆          ┆   ┆ 5         ┆         

'\nenriched_sales = (\n    sales\n    .join(\n        customers.select(["customer_id", "customer_name"]),\n        on="customer_id",\n        how="left"\n    )\n    .join(\n        products.select(["product_id", "product_name"]),\n        on="product_id",\n        how="left"\n    )\n    .join(\n        stores.select(["store_id", "store_name"]),\n        on="store_id",\n        how="left"\n    )\n    .select(\n        "sale_id",\n        "customer_name",\n        "product_name",\n        "store_name",\n        "quantity"\n    )\n)\n'